In [2]:
from llmsherpa.readers import LayoutPDFReader
import html2text
from docx import Document
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTPage
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
import textwrap
import re

import os
import pandas as pd
ed_path = os.path.join(os.getcwd(), "regjeringen", "ed")
oljedir_tildelingsbrev  = os.path.join(ed_path, "tildelingsbrev-til-oljedirektoratet-for-2022.pdf")
oljedir_tildelingsbrev


llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all" #APIET TIL llmsherpa, snart decommisioned
# llmsherpa_api_url = "http://localhost:5001/api/parseDocument?renderFormat=all" # Eget api gjennom docker
pdf_url = os.path.join(ed_path, "tildelingsbrev-til-oljedirektoratet-for-2022.pdf")

# Lager en PDFReader/doc    
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# Leser inn pdfen/Doc
doc = pdf_reader.read_pdf("Rapporter - Original/27_innovative_loesninger_-_medicoindustrien.pdf")

doc



In [3]:
html_string = ""


for i in range(0, len(doc.sections()), 1):
        # print(section.title)
        html_string += doc.sections()[i].title
        # print(f"\ntitle: {section.title}")
        html_string += doc.sections()[i].to_html(include_children=True, recurse=False)
        # print(doc.sections()[i].to_html(include_children=True, recurse=False))


# Initialize html2text converter
text_maker = html2text.HTML2Text()
# Convert HTML to Markdown
markdown_output = text_maker.handle(html_string)

In [4]:
markdown_output

'27 innovative løsninger\n\n# 27 innovative løsninger\n\nMedicoindustriens eksempelsamling på effektiviseringsfremmende og\narbejdskraftbesparende teknologier\n\n## Indhold\n\n## Indledning\n\n## Apodan\n\n## Apodan\n\n## Boston Scientific\n\n## Boston Scientific\n\n## Codan DEHA\n\n## Cook Medical\n\n## Duomed Scandinavia\n\n## Intuitive\n\n## Johnson & Johnson\n\n## Medtronic\n\n## Medtronic\n\n## Medtronic\n\n## Olympus\n\n## Olympus\n\n## Olympus\n\n## Onemed\n\n## Pro Devices A/S\n\n## Siemens Healthineers\n\n## Siemens Healthineers\n\n## Siemens Healthineers\n\n## Treat Systems\n\n## Wear&Care;\n\nIndhold\n\n## Indhold\n\nIndledning| 3  \n---|---  \nApodan: Lympha Press®| 4  \nApodan: Polymem®| 6  \nBoston Scientific: Heart Logic8 Boston Scientific: Rezum| 11  \nBoston Scientific: WATCHMAN FLX| 14  \nCodan DEHA: Mix-Ad Set med GUARD®| 18  \nCook Medical: Zilver PTX| 20  \nDuomed Scandinavia: SB System Kapselendoskopi| 21  \nFluoGuide: Oplysning af cancer-celler| 23  \nIntuitive: 

In [5]:
with open("helse.md", "w") as file:
    file.write(markdown_output)


In [6]:
import requests
import time
import os
from openai import AzureOpenAI

def api_call_gpt_with_retry(message_content:str, max_retries=3, delay=2):

    client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-05-15"
    )

    system_message = f'''
    Du er en fantaistisk oversetter fra Dansk til Norsk bokmpl.
    Du oversetter danske konsulentrapporter til norsk, så språket ditt bør være direkte og profesjonelt.
    IKKE INKLUDER ekstra tekst, kun oversett ordene som er gitt i oppgaven direkte til Norsk.
    '''

    prompt = f'''Oversett den følgende teksten fra Dansk til Norsk: {message_content}\n'''
    messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ]
    

    for attempt in range(max_retries): 
        response = client.chat.completions.create(
            model= os.getenv("MODEL_NAME"), # model = "deployment_name".
            messages=messages)

        try:
        
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error {e}")
            print(response.text)
            if attempt < max_retries - 1:
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries reached. Exiting.")

In [7]:
# Splitter filen.
headers_to_split_on = [("#", "Kapittel"), ("##", "Underkapittel 1"), ("###", "Underkapittel 2"), ("####", "Underkapittel 3"), ("#####", "Underkapittel 4"), ("######", "Underkapittel 5"), ("######", "Underkapittel 6")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)


splitted_file = markdown_splitter.split_text(markdown_output)

threshold = 200

def tekstlengde_sjekk(document):
    return len(document.page_content) >= threshold

filtered_documents = list(filter(tekstlengde_sjekk, splitted_file))
filtered_documents

[Document(page_content='Indledning| 3\n---|---\nApodan: Lympha Press®| 4\nApodan: Polymem®| 6\nBoston Scientific: Heart Logic8 Boston Scientific: Rezum| 11\nBoston Scientific: WATCHMAN FLX| 14\nCodan DEHA: Mix-Ad Set med GUARD®| 18\nCook Medical: Zilver PTX| 20\nDuomed Scandinavia: SB System Kapselendoskopi| 21\nFluoGuide: Oplysning af cancer-celler| 23\nIntuitive: Da Vinci Surgical System| 24\nJohnson & Johnson: Antibakteriel sutur| 27\nMagVenture: Transkraniel magnet stimulation (TMS)| 29\nMedtronic: CADx| 32\nMedtronic: LINQTM Implantable Cardiac Monitor (ICM)| 35\nMedtronic: Care Connect – Remote Management of Diabetes| 39\nOlympus: Move| 41\nOlympus: EndoCuff Vision| 43\nOlympus: Orbeye| 44\nOnemed: Swash®| 46\nPro Devices A/S: PD-100, PD-Auto & PD-Technology| 48\nResMed: Airview| 51\nSiemens Healthineers: AI-Rad| 53\nSiemens Healthineers: Syngo Virtual Cockpit (sVC)| 55\nSiemens Healthineers: Migo| 57\nTreat Systems: TREAT-essential| 60\nWear&Care;: Bleer baseret på sensorteknolg

In [8]:
# Import necessary modules
import os
from dotenv import load_dotenv
from pathlib import Path

# Get the folder path of the Jupyter Notebook file
notebook_path = os.path.abspath("__file__")
parent_folder = os.path.dirname(notebook_path)
project_folder = os.path.join(parent_folder, "Rapporter - Original")
ferdig_dokumenter_path = os.path.join(parent_folder,"Maskinoversatte sammendrag")

# Set the name of the Word document to translate
dokumentnavn = r"Sjeldne diagnoser og prioritering notat utkast Menon 9 februar 2023"

# Create the full path to the Word document
doc_to_translate = os.path.join(project_folder, f"{dokumentnavn}.docx")

# Set the path to the folder where translated documents will be saved


# Path to your .env file (replace with the actual path)
env_path = os.path.join(parent_folder, '.env')

# Load the .env file to get the OpenAI API key
load_dotenv(dotenv_path=env_path)
api_key = os.getenv('OPENAI_API_KEY')



In [9]:
print(api_call_gpt_with_retry("Hei, hvordan går det?"))

Hei, hvordan går det?


In [10]:
new_doc = Document()
i = 0
lengde = len(filtered_documents)

for doc in filtered_documents[1:-1]:
    i += 1
    print(f"Translating text {i} of {lengde}")
    tekst = doc.page_content.replace("\n", " ")
    translated_text =  api_call_gpt_with_retry(tekst)
    new_doc.add_heading(list(doc.metadata.items())[-1][-1])
    new_doc.add_paragraph(translated_text)

new_doc.save(f"medisin_for_capp_full.docx")

Translating text 1 of 137
Translating text 2 of 137
Translating text 3 of 137
Translating text 4 of 137
Translating text 5 of 137
Translating text 6 of 137
Translating text 7 of 137
Translating text 8 of 137
Translating text 9 of 137
Translating text 10 of 137
Translating text 11 of 137
Translating text 12 of 137
Translating text 13 of 137
Translating text 14 of 137
Translating text 15 of 137
Translating text 16 of 137
Translating text 17 of 137
Translating text 18 of 137
Translating text 19 of 137
Translating text 20 of 137
Translating text 21 of 137
Translating text 22 of 137
Translating text 23 of 137
Translating text 24 of 137
Translating text 25 of 137
Translating text 26 of 137
Translating text 27 of 137
Translating text 28 of 137
Translating text 29 of 137
Translating text 30 of 137
Translating text 31 of 137
Translating text 32 of 137
Translating text 33 of 137
Translating text 34 of 137
Translating text 35 of 137
Translating text 36 of 137
Translating text 37 of 137
Translatin